In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import alpaca_trade_api as tradeapi

# API keys
alpaca_api_key = "PKJYB527YAQVS4J907RJ"
alpaca_secret_key = "zb5bvTpu8cWaKIgAbDCESouV9MX0qn4RXiQzS5n8"

# Create the Alpaca API object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

# Set the tickers and timeframe
tickers = ["TSLA"]
timeframe = "1Day"

# Set the date range
start = pd.Timestamp("2014-02-07", tz="America/New_York").isoformat()
end = pd.Timestamp("2020-02-07", tz="America/New_York").isoformat()

# Get the data
df = alpaca.get_bars(tickers, timeframe, start=start, end=end).df

# Drop the 'symbol' column and add shifted close column
df = df.drop(columns=['symbol'])

# Function to create the signal
def signal_creator_5(row):
    percent_diff = (row['close'] - row['shifted_close_5']) / row['shifted_close_5']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

def signal_creator_15(row):
    percent_diff = (row['close'] - row['shifted_close_15']) / row['shifted_close_15']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

def signal_creator_30(row):
    percent_diff = (row['close'] - row['shifted_close_30']) / row['shifted_close_30']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

# 5 day shift
df['shifted_close_5'] = df['close'].shift(periods=5)
df['signal_5'] = 0
print(df)
df['signal_5'] = df.apply(signal_creator_5, axis=1)

# 15 day shift
df['shifted_close_15'] = df['close'].shift(periods=15)
df['signal_15'] = 0
print(df)
df['signal_15'] = df.apply(signal_creator_15, axis=1)

# 30 day shift
df['shifted_close_30'] = df['close'].shift(periods=30)
df['signal_30'] = 0
df['signal_30'] = df.apply(signal_creator_30, axis=1)

# Columns and NA drops to prevent data leakege
df.dropna(inplace=True)
df.drop(columns=['shifted_close_5','shifted_close_15','shifted_close_30'],inplace=True)

# Separate features and target
features = df.drop(columns = ['signal_5', 'signal_15', 'signal_30'], axis=1)
target = df.drop(columns = ['close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
print (target)

In [ ]:
# Scale the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# Function to create dataset for LSTM
def create_dataset(X, y, time_steps=5):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps), :])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Creating the dataset
time_steps = 5
X, y = create_dataset(scaled_features, target.values, time_steps)

# Splitting data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Building the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Model Evaluation
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Making Predictions
predictions = model.predict(X_test)

# Function to interpret predictions
def interpret_prediction(prediction):
    if prediction > 0.5:
        return 1  # Buy
    elif prediction < -0.5:
        return -1  # Sell
    else:
        return 0  # Hold

interpreted_signals = [interpret_prediction(p[0]) for p in predictions]

# Example: Print first 10 decisions
print(interpreted_signals[:10]) 